In [1]:
import torch.nn as nn
from model_numba.Layers.Layers import batchNorm2D, Convolution2D_GPU
import numpy as np
from torchvision import transforms
from model_numba.unet_cuda import Double_Conv, Unet_Cuda
from PIL import Image
import torch
import time
from numba import cuda

In [2]:
transform = transforms.Compose(
    [
        transforms.Resize((512, 512)),
        transforms.ToTensor()
    ]
)
image = Image.open('test.jpg')
image = transform(image).unsqueeze(0)
image.shape

torch.Size([1, 3, 512, 512])

In [3]:
from model.unet import U_net
model = U_net(1).cuda().eval()
weight = torch.load('weights/weights.pth')
model.load_state_dict(weight)

numba_model = Unet_Cuda(1)

In [4]:

start = time.time()
out = model(image.cuda())
end = time.time()

print(f'Torch Runtime: {end- start} s')

start = time.time()
out_numba = numba_model(image.cpu().detach().numpy())
end = time.time()

print(f'Numba Runtime: {end- start} s')

mean = np.mean(np.abs(model.d5.cpu().detach().numpy()- numba_model.d5.copy_to_host()))
max = np.max(np.abs(model.d5.cpu().detach().numpy()- numba_model.d5.copy_to_host()))
print(f'Mean Difference: {mean : .6f}')
print(f'Max Difference: {max: .6f}')

Torch Runtime: 0.2398977279663086 s
Numba Runtime: 0.8706598281860352 s
Mean Difference:  0.000045
Max Difference:  0.001294
